In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from implicit.nearest_neighbours import ItemItemRecommender
from implicit.evaluation import train_test_split, mean_average_precision_at_k

print("Semua library berhasil di-import.")
file_path_lokal = r'D:\Sem 7\Sistem Rekomendasi\proyek-sistem-informasi\data-books\train.csv'
file_path_lokal = file_path_lokal.strip().strip('"')

try:
    df = pd.read_csv(file_path_lokal)
    print(f"\nData '{file_path_lokal}' berhasil dimuat.")
    
    df['user_id_cat'] = df['user_id'].astype("category")
    df['item_id_cat'] = df['item_id'].astype("category")
    
    n_users = len(df['user_id_cat'].cat.categories)
    n_items = len(df['item_id_cat'].cat.categories)
    # Pastikan angka ini sesuai dengan error Anda:
    print(f"Users: {n_users}, Items: {n_items}") 

    # ===== INI ADALAH PERBAIKAN KRUSIAL =====
    # Pastikan (baris, kolom) adalah (user, item)
    interactions_matrix = sparse.csr_matrix(
        (np.ones(df.shape[0]),
         (df['user_id_cat'].cat.codes,  # <-- Baris = USER
          df['item_id_cat'].cat.codes) # <-- Kolom = ITEM
        ),
        shape=(n_users, n_items) # <-- Shape = (Users, Items)
    )
    # =======================================
    
    print(f"Shape Matriks: {interactions_matrix.shape}") # <-- HARUS (Users, Items)
    
    train_matrix, test_matrix = train_test_split(interactions_matrix, train_percentage=0.8, random_state=42)
    print(f"Shape Train Matrix: {train_matrix.shape}") # <-- HARUS (Users, Items)

    print("\nMulai melatih model baseline (IBCF)...")
    model_baseline = ItemItemRecommender(K=20)
    model_baseline.fit(train_matrix) # <-- Benar: (Users, Items)
    print("Model baseline (IBCF) selesai dilatih.")

    print("\nMulai menghitung skor MAP@10 untuk model baseline...")
    map10_baseline = mean_average_precision_at_k(model_baseline,
                                                 train_matrix, # <-- Benar: (Users, Items)
                                                 test_matrix,  # <-- Benar: (Users, Items)
                                                 K=10,
                                                 num_threads=0) 

    print("\n--- HASIL MODEL BASELINE (IBCF) ---")
    print(f"Skor MAP@10: {map10_baseline}")

except Exception as e:
    print(f"\nTerjadi error di Sel 1: {e}")

Semua library berhasil di-import.

Data 'D:\Sem 7\Sistem Rekomendasi\proyek-sistem-informasi\data-books\train.csv' berhasil dimuat.
Users: 13876, Items: 123069
Shape Matriks: (13876, 123069)
Shape Train Matrix: (13876, 123069)

Mulai melatih model baseline (IBCF)...


  0%|          | 0/123069 [00:00<?, ?it/s]

Model baseline (IBCF) selesai dilatih.

Mulai menghitung skor MAP@10 untuk model baseline...


  0%|          | 0/11454 [00:00<?, ?it/s]


--- HASIL MODEL BASELINE (IBCF) ---
Skor MAP@10: 0.018602178412511838
